In [1]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
#! pip install xmltodict

In [2]:
p = beam.Pipeline(InteractiveRunner())

In [3]:
from google.cloud import storage
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "gcp-bq-2021"
bucket=storage_client.get_bucket(bucket_name)
blobs=list(bucket.list_blobs(prefix="xmlfiles/"))

blob_files = [blob.name for blob in blobs if ".xml" in blob.name]

In [4]:
table_schema = {
    "fields": [
        {'name' : 'CustomerID', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'EmployeeID', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'OrderDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'RequiredDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'ShipInfo', 'type': 'RECORD', 'mode': 'NULLABLE', 'fields': [
            {'name' : 'ShipVia', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'Freight', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipName', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipAddress', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipCity', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipRegion', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipPostalCode', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipCountry', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShippedDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        ]},
    ]
}
        

In [5]:
def readfiles(element):
    import xmltodict
    bucket_name = "gcp-bq-2021"
    bucket=storage_client.get_bucket(bucket_name)
    blob = bucket.get_blob(element)
    parsed_xml = xmltodict.parse(blob.download_as_string())
    #print(parsed_xml)
    return parsed_xml

def printelement(element):
    print(element)
    print("#####")


def formatting(order):
    import copy
    order_copy = copy.deepcopy(order)
    #print(order)
    if "@ShippedDate" in order['ShipInfo']:
        order_copy['ShipInfo']['ShippedDate'] = order['ShipInfo']['@ShippedDate']
        del order_copy['ShipInfo']['@ShippedDate']
    # code 3 order_copy["CustomerIDEmployeeID"] = order["CustomerID"].lower() #+ "-" + order["EmployeeID"]
    # del order_copy["CustomerID"]
    #del order_copy["EmployeeID"]
    #print(order_copy)
    return order_copy
    
def xmlformatting(element):
    for order in element['Root']['Orders']['Order']:
        #print(order)
        yield formatting(order)

# code1 - static table name
#pipeline_data = (p | beam.Create(blob_files) | beam.Map(readfiles) | beam.FlatMap(xmlformatting) |
#                 beam.io.WriteToBigQuery("gcp-bq-2021:dataset1.table3",
#                                       schema=table_schema,
#                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND, #WRITE_TRUNCATE
#                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
#                       custom_gcs_temp_location="gs://gcp-bq-2021"))

# code 2 - dynamic table name from the xml column
#pipeline_data = (p | beam.Create(blob_files) | beam.Map(readfiles) | beam.FlatMap(xmlformatting) |
#                 beam.io.WriteToBigQuery(table= lambda row: "gcp-bq-2021:dataset1." + row['CustomerIDEmployeeID'],
#                                       schema=table_schema,
#                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND, #WRITE_TRUNCATE
#                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
#                       custom_gcs_temp_location="gs://gcp-bq-2021"))

# code 3 function
def tablename(e):
    print("######%%")
    print(e)
    import copy
    element = copy.deepcopy(e)
    a = e["CustomerIDEmployeeID"]
    del e["CustomerIDEmployeeID"]
    return "gcp-bq-2021:dataset1." + a

# code 3- Dynamic table name from xml column and deleting the column (the "CustomerIDEmployeeID" column name should 
# be romoved from the schema
#pipeline_data = (p | beam.Create(blob_files) | beam.Map(readfiles) | beam.FlatMap(xmlformatting) |
#                 beam.io.WriteToBigQuery(table = lambda row: tablename(row),
#                                       schema=table_schema,
#                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND, #WRITE_TRUNCATE
#                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
#                       custom_gcs_temp_location="gs://gcp-bq-2021"))

# code 4 - Dynamic table name same as that of input xml file by adding file name as key in the dictionary and accessing
# them in writetobigquery

#code 4 - Read files function
def readfiles(element):
    import xmltodict
    bucket_name = "gcp-bq-2021"
    bucket=storage_client.get_bucket(bucket_name)
    blob = bucket.get_blob(element)
    parsed_xml = xmltodict.parse(blob.download_as_string())
    #print(parsed_xml)
   # print(element.split("/")[1].split(".")[0])
    return parsed_xml, element.split("/")[1].split(".")[0]

# code 4 - formatting
def formatting(order, filename):
    import copy
    order_copy = copy.deepcopy(order)
    #print(order)
    if "@ShippedDate" in order['ShipInfo']:
        order_copy['ShipInfo']['ShippedDate'] = order['ShipInfo']['@ShippedDate']
        del order_copy['ShipInfo']['@ShippedDate']
    # code 3 order_copy["CustomerIDEmployeeID"] = order["CustomerID"].lower() #+ "-" + order["EmployeeID"]
    # del order_copy["CustomerID"]
    #del order_copy["EmployeeID"]
    #print(order_copy)
    order_copy['filename'] = filename
    return order_copy
    
# code 4 - xmlformatting
def xmlformatting(element):
    print("%%%%%%%%")
    data, filename = element
    for order in data['Root']['Orders']['Order']:
        #print(order)
        yield formatting(order, filename)


# code 4 - tablename
def tablename(e):
    print("######%%")
    print(e)
    import copy
    element = copy.deepcopy(e)
    a = e["filename"]
    del e["filename"]
    return "gcp-bq-2021:dataset1." + a

pipeline_data = (p | beam.Create(blob_files) | beam.Map(readfiles) | beam.FlatMap(xmlformatting) |
                 beam.io.WriteToBigQuery(table = lambda row: tablename(row),
                                       schema=table_schema,
                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND, #WRITE_TRUNCATE
                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                       custom_gcs_temp_location="gs://gcp-bq-2021"))
                 
                 #beam.ParDo(dynamicbigquerywrite))



p.run()

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2138: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  is_streaming_pipeline = p.options.view_as(StandardOptions).streaming
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:1128: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


%%%%%%%%
######%%
OrderedDict([('CustomerID', 'xml1GREAL'), ('EmployeeID', '6'), ('OrderDate', '1997-05-06T00:00:00'), ('RequiredDate', '1997-05-20T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '3.35'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-05-09T00:00:00')])), ('filename', 'xml1')])
######%%
OrderedDict([('CustomerID', 'xml1GREAL'), ('EmployeeID', '8'), ('OrderDate', '1997-07-04T00:00:00'), ('RequiredDate', '1997-08-01T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '4.42'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-07-14T00:00:00')])), ('filename', 'xml1')])
######%%
OrderedDict([('CustomerID', 'xml1GREAL'), ('EmployeeID', '1'), (

In [3]:
# import xml files from gcs



def readfilexml(element):
    print("##")
    import xmltodict
    gcs = beam.io.gcp.gcsio.GcsIO()
    file = gcs.open(filename=element)
    print("type: {}".format(type(file)))
    parsed_xml = xmltodict.parse(file.read())
    print(parsed_xml)
    #return parsed_xml

a = p | beam.Create(["gs://gcp-bq-2021/xmlfiles/xml1.xml"]) | beam.ParDo(readfilexml)

p.run()



##
type: <class '_io.BufferedReader'>
OrderedDict([('Root', OrderedDict([('@xmlns', 'http://www.adventure-works.com'), ('Orders', OrderedDict([('Order', [OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '6'), ('OrderDate', '1997-05-06T00:00:00'), ('RequiredDate', '1997-05-20T00:00:00'), ('ShipInfo', OrderedDict([('@ShippedDate', '1997-05-09T00:00:00'), ('ShipVia', '2'), ('Freight', '3.35'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA')]))]), OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '8'), ('OrderDate', '1997-07-04T00:00:00'), ('RequiredDate', '1997-08-01T00:00:00'), ('ShipInfo', OrderedDict([('@ShippedDate', '1997-07-14T00:00:00'), ('ShipVia', '2'), ('Freight', '4.42'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA')]

In [ ]:
# PARSING XML USING LXML

In [6]:
! pip install lxml

     |████████████████████████████████| 6.4 MB 7.8 MB/s            


In [12]:
from lxml import etree

def parse_xml(xmlFile):
    """
    Parse the xml file
    """
    
    with open(xmlFile) as fileobj:
        xml = fileobj.read()
    
    print("xml after io read: {}".format(xml))
    root = etree.fromstring(xml)
    
    print("xml is: {}".format(root))
    
    for aptt in root.getchildren():
        for element in aptt.getchildren():
            if not element.text:
                text = "None"
            else:
                text = element.text
            print(element.tag+"->"+text)

if __name__ == "__main__":
    parse_xml("example.xml")

xml after io read: <?xml version="1.0" ?>
<zAppointments reminder="15">
    <appointment>
        <begin>1181251680</begin>
        <uid>040000008200E000</uid>
        <alarmTime>1181572063</alarmTime>
        <state></state>
        <location></location>
        <duration>1800</duration>
        <subject>Bring pizza home</subject>
    </appointment>
    <appointment>
        <begin>1234360800</begin>
        <duration>1800</duration>
        <subject>Check MS Office website for updates</subject>
        <location></location>
        <uid>604f4792-eb89-478b-a14f-dd34d3cc6c21-1234360800</uid>
        <state>dismissed</state>
  </appointment>
</zAppointments>
xml is: <Element zAppointments at 0x7f1196cf71e0>
begin->1181251680
uid->040000008200E000
alarmTime->1181572063
state->None
location->None
duration->1800
subject->Bring pizza home
begin->1234360800
duration->1800
subject->Check MS Office website for updates
location->None
uid->604f4792-eb89-478b-a14f-dd34d3cc6c21-1234360800
state->d

In [8]:
# use objectify module to read as python objects
from lxml import etree, objectify

with open("example.xml") as fileobj:
    xml = fileobj.read()

root = objectify.fromstring(xml)
#print("root: {}".format(root))
def elem2dict(node):
    """
    Convert an lxml.etree node tree into a dict.
    """
    result = {}

    for element in node.iterchildren():
        # Remove namespace prefix
        key = element.tag.split('}')[1] if '}' in element.tag else element.tag

        # Process element as tree element if the inner XML contains non-whitespace content
        if element.text and element.text.strip():
            value = element.text
        else:
            #print(element.tag)
            value = elem2dict(element)
        if key in result:

            
            if type(result[key]) is list:
                result[key].append(value)
            else:
                #print(value)
                tempvalue = result[key].copy()
                result[key] = [tempvalue, value]
        else:
            #print(value)
            result[key] = value
    return result

a=elem2dict(root)
a

{'begin': '1234360800', 'duration': '1800', 'subject': 'Check MS Office website for updates', 'location': {}, 'uid': '604f4792-eb89-478b-a14f-dd34d3cc6c21-1234360800', 'state': 'dismissed'}


{'appointment': [{'begin': '1181251680',
   'uid': '040000008200E000',
   'alarmTime': '1181572063',
   'state': {},
   'location': {},
   'duration': '1800',
   'subject': 'Bring pizza home'},
  {'begin': '1234360800',
   'duration': '1800',
   'subject': 'Check MS Office website for updates',
   'location': {},
   'uid': '604f4792-eb89-478b-a14f-dd34d3cc6c21-1234360800',
   'state': 'dismissed'}]}

In [ ]:
import pandas